In [1]:
%run "..\..\Startup_py3.py"
sys.path.append(r"..\..\..\..\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

import h5py
from ImageAnalysis3.classes import _allowed_kwds
import ast

28720


## load codebook

In [2]:
import pandas as pd
codebook_filename = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-11_brain\Summary_tables\CTP11-mouse-genome-1000_codebook.csv'
#
codebook_df = pd.read_csv(codebook_filename, header=0)
codebook_df

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_824,NDB_712,NDB_818,NDB_940,NDB_872,NDB_934,NDB_926,NDB_910,chr,chr_order
0,1:3740000-3760000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1:6240000-6260000,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,1:8740000-8760000,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
3,1:11240000-11260000,4,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,3
4,1:13740000-13760000,5,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983,X:161240000-161260000,1057,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,X,52
984,X:163740000-163760000,1058,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,X,53
985,X:166240000-166260000,1059,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,X,54
986,X:168740000-168760000,1060,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,55


## load spot files

In [4]:

spot_file_folder = r'\\crick\SSD_1\Pu_Temp\20211220-P_brain_CTP11-1000_CTP12-DNA_from_1216\CandSpots'

spot_filenames = [os.path.join(spot_file_folder, _fl) for _fl in os.listdir(spot_file_folder)]
# remove first 2 fov for now
#spot_filenames = [_fl for _fl in spot_filenames if int(_fl.split('Conv_zscan_')[1].split('_')[0]) >= 27]

In [5]:
import multiprocessing as mp
from ImageAnalysis3.classes import decode
reload(decode)
decode_args = [(_fl, codebook_df) for _fl in spot_filenames[:]
               if not os.path.exists(_fl.replace('CandSpots', 'Decoder'))]
print(len(decode_args))

5302


In [ ]:
%%time
with mp.Pool(44) as drift_pool:
    align_results = drift_pool.starmap(decode.batch_decode_DNA, decode_args[:], chunksize=1)
    drift_pool.close()
    drift_pool.join()
    drift_pool.terminate()